## Training Section

### Preprocess   

* tfs --- TF with condition values as row
* genes -- genes to fit data

In [4]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('dataset/expression/KO.txt', sep = '\t').set_index('Gene')

In [3]:
tf_names = pd.read_csv ('dataset/expression/KO_TF_names.txt',sep = '\t')

In [4]:
tfs = dataset.loc[tf_names.values.flatten()]
genes = dataset.drop(tf_names.values.flatten())

### LASSO Regression

In [3]:
from sklearn import model_selection
from sklearn.linear_model import Lasso, LassoCV

In [26]:
def lasso_regression(gene, tfs):
    # prepare data
    x = tfs.values.transpose()
    y = gene.values
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.1)
    
    # lasso train
    lasso_reg = LassoCV(alphas = [0.01, 0.1, 1, 10, 100], cv=5)
    lasso_reg.fit(x_train, y_train)

    #filter out zero coef
    predictions = tfs[lasso_reg.coef_ != 0]
    coef = lasso_reg.coef_[lasso_reg.coef_ != 0]
    return predictions.index,coef

In [35]:
lasso_infer = open("dataset/inferred/LASSO.txt","w") 
for i in range(len(genes)): 
    target_gene = genes.index[i]
    predictions,coef = lasso_regression(genes.iloc[i],tfs)
    for j  in range(len(predictions)):
        prediction = str(predictions[j])
        score = str(coef[j])
        lasso_infer.write(target_gene + ' ' + prediction + ' '+ score + '\n')
lasso_infer.close()

C:\Users\tonyt\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2812924359706841, tolerance: 0.12397375466886684
  positive)
C:\Users\tonyt\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5261262030816525, tolerance: 0.06798389501002794
  positive)
C:\Users\tonyt\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.12109382625790488, tolerance: 0.06652559795591602
  positive)
C:\Users\tonyt\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\coo

## Evaluation Section

## GENIE3

### The inferred result from our method:

In [28]:
inferred_result = np.genfromtxt('dataset/inferred/LASSO.txt', names=['gene_a','gene_b','confidence'], dtype=None)

C:\Users\tonyt\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [29]:
all_possible_pairs = np.genfromtxt('dataset/inferred/LASSO.txt', names=['gene_a','gene_b'], dtype=None,usecols=(0,1))

C:\Users\tonyt\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [30]:
inferred_result_binary = np.genfromtxt('dataset/inferred/LASSO.txt',usecols=(2)).flatten()

### The ground truth:

In [31]:
gold_standard = np.genfromtxt('dataset/gold/YEASTRACT_Count3.KO.txt',names=['gene_a','gene_b'],dtype=None)

C:\Users\tonyt\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [34]:
gold_standard_set = set([ e[0]+e[1] for e in gold_standard]).union(set([ e[1]+e[0] for e in gold_standard]))

In [32]:
def in_array(e,gold_standard_set):
    if e[0]+e[1] in gold_standard_set:
        return True
    return False

In [33]:
def match_array(e,gold_standard_set):
    if in_array(e,gold_standard_set):
        return 1
    else:
        return 0

In [35]:
gold_standard_binary = np.array([match_array(e,gold_standard_set) for e in all_possible_pairs])

In [36]:
gold_standard_binary.sum()

658

### auc roc:

In [37]:
from sklearn import metrics

In [38]:
fpr, tpr, thresholds = metrics.roc_curve(gold_standard_binary, inferred_result_binary)

In [39]:
auc = metrics.roc_auc_score(gold_standard_binary, inferred_result_binary)

In [0]:
import matplotlib.pyplot as plt

In [16]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange',
        label='ROC curve (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('GENIE3')
plt.legend(loc="lower right")
plt.show()

NameError: name 'plt' is not defined